In [ ]:
import pandas as pd
import random

In [ ]:
!pip install surprise

In [ ]:
from surprise import SVDpp
from surprise import model_selection
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

In [ ]:
ratings_pd = pd.read_csv('ratings.csv')
ratings_pd.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings_pd.shape

(100836, 4)

In [ ]:
ratings_pd.movieId.unique().shape[0]

9724

In [ ]:
ratings_pd.userId.unique().shape[0]

610

In [ ]:
min_r = ratings_pd.rating.min()
max_r = ratings_pd.rating.max()
print(min_r, ' - ', max_r)

0.5  -  5.0


In [ ]:
#Izbrisati posle
ratings_pd.rating.unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [ ]:
ratings_pd.movieId.unique().shape[0]

9724

In [ ]:
reader = Reader()
ratings = Dataset.load_from_df(ratings_pd[['userId', 'movieId', 'rating']], reader = reader)


In [ ]:
parameters = {'n_epochs' : [15], 'n_factors':[10, 20], 'lr_all': [0.005, 0.01], 'reg_all': [0.02, 0.05], 'verbose' : [True]}

In [ ]:
grid_search = model_selection.GridSearchCV(SVDpp, param_grid=parameters, measures=['rmse'], return_train_measures=True, cv = 5)
grid_search.fit(ratings)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 proc

In [ ]:
grid_search.best_params

{'rmse': {'lr_all': 0.01,
  'n_epochs': 15,
  'n_factors': 20,
  'reg_all': 0.05,
  'verbose': True}}

In [ ]:
grid_search.best_score

{'rmse': 0.8593970936492008}

In [ ]:
svdpp = grid_search.best_estimator['rmse']
train = ratings.build_full_trainset()
svdpp.fit(train)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14


In [ ]:
predictions_train = svdpp.test(train.build_testset())
accuracy.rmse(predictions_train)

RMSE: 0.7515


0.7515260084283178

In [ ]:
test = train.build_anti_testset()
predictions = svdpp.test(test)
accuracy.rmse(predictions)

RMSE: 0.5065


0.5064594350010255

In [ ]:
#Ovo je da bih videla koje parametre predikcija ima
predictions[0]

Prediction(uid=1, iid=318, r_ui=3.501556983616962, est=5, details={'was_impossible': False})

In [ ]:
def recommended_movies(pred, user, n):
  
  best_movies = []

  for uid, iid, _, est, _ in pred:
    if uid == user:
      best_movies.append((iid, est))
  
  best_movies.sort(key = lambda k: k[1], reverse = True)

  best_movies = best_movies[:n]

  movies_pd = pd.read_csv('movies.csv')

  recommendation = pd.DataFrame()

  i = 0

  for m, _ in best_movies:
    recommendation = recommendation.append(movies_pd.loc[movies_pd['movieId'] == m])
  
  recommendation = recommendation.reset_index()

  recommendation = recommendation.drop(['index'], axis = 1)

  return recommendation

In [ ]:
recommended_movies(predictions, 6, 5)

,movieId,title,genres
0,2542,"Lock, Stock & Two Smoking Barrels (1998)",Comedy|Crime|Thriller
1,1248,Touch of Evil (1958),Crime|Film-Noir|Thriller
2,3451,Guess Who's Coming to Dinner (1967),Drama
3,1204,Lawrence of Arabia (1962),Adventure|Drama|War
4,7371,Dogville (2003),Drama|Mystery|Thriller


In [ ]:
recommended_movies(predictions, 27, 5)

,movieId,title,genres
0,7371,Dogville (2003),Drama|Mystery|Thriller
1,1204,Lawrence of Arabia (1962),Adventure|Drama|War
2,904,Rear Window (1954),Mystery|Thriller
3,56782,There Will Be Blood (2007),Drama|Western
4,3266,Man Bites Dog (C'est arrivé près de chez vous)...,Comedy|Crime|Drama|Thriller


In [ ]:
recommended_movies(predictions, 196, 5)

,movieId,title,genres
0,3451,Guess Who's Coming to Dinner (1967),Drama
1,1217,Ran (1985),Drama|War
2,318,"Shawshank Redemption, The (1994)",Crime|Drama
3,1104,"Streetcar Named Desire, A (1951)",Drama
4,177593,"Three Billboards Outside Ebbing, Missouri (2017)",Crime|Drama
